In [ ]:
import csv

import os
os.environ["VOCAB"] = "placeholder"
os.environ["CORPORA"] = "placeholder"
os.environ["ISO_LANG"] = "en"

from stemmers import stemmers
# from create import tokenize_values

In [ ]:
fname = "../dict/mfd2.0.dic"  # Hopp, MFT
# fname = "../dict/mft_original.dic"  # multilabel
# fname = "../dict/Provisional_dictionary.dic"
# fname = "../dict/Refined_dictionary.dic" # Ponizovskiy, Schwartz

In [ ]:
# phases: 0 - init; 1 - classes; 2 - vocab
phases = 0
classes = {}
codes = {}

with open(fname) as fin:
    src = csv.reader(fin, delimiter="\t")
    for r in src:
        if not r:
            continue
        if r[0] == "%":
            phases += 1
            continue
        if phases == 1:
            classes[r[0]] = r[1]
            codes[r[0]] = []
            continue
        assert phases == 2, f"Not expected to arrive to phase {phases}"
        if " " not in r[0]:
            codes[r[1]] += [r[0]]
classes

In [ ]:
for s in stemmers.keys():
    backref = {}
    for c, r in codes.items():
        for v in r:
            stem = stemmers[s](v, None)
            if stem in backref:
                backref[stem] |= set([c])
            else:
                backref[stem] = set([c])
    fname_out = fname.replace(".dic", f".{s}.csv")
    print(fname_out)
    print(backref)
    with open(fname_out, "w") as fout:
        dest = csv.writer(fout)
        for c, r in codes.items():
            row = [classes[c].replace(".", "_")] + [
                v for v in r if len(backref[stemmers[s](v, None)]) == 1
            ]
            dest.writerow(row)